## 3-3. How to use QURI Parts
QURI Parts is an open source library suite for creating and executing quantum algorithms on various quantum computers and simulators. In this section, you will learn how to install and use basic functions provided by QURI Parts.


### Covered areas and components
1.<b>Core components:</b><br>`quri-parts-circuit`: Quantum circuit (Gate, circuit, noise etc.)<br>`quri-parts-core`: General components (Operator, state, estimator, sampler etc.)<br>2.<b>Platform (device/simulator) support:</b> <br><b>Quantum circuit simulators:</b><br>`quri-parts-qulacs`: Qulacs <br>`quri-parts-stim`: Stim <br>`quri-parts-itensor`: ITensor <br><b>Quantum platforms/SDKs:</b> <br>`quri-parts-braket`: Amazon Braket SDK <br>`quri-parts-cirq`: Cirq (Only circuit conversion is supported yet) <br>`quri-parts-qiskit`: Qiskit (Circuit conversion and execution are not supported yet) <br>3. <b>Intermediate representation support:</b><br>`quri-parts-openqasm`: OpenQASM 3.0 <br>`quri-parts-algo`: Algorithm (Ansatz, optimizer, error mitigation etc.)<br><b>Chemistry</b><br>`quri-parts-chem`: General concepts, Fermion-qubit mapping etc.<br><b>Library support:</b> <br>`quri-parts-openfermion` <br> ### Installation of QURI Parts 
 QURI Parts requires Python 3.9.8 or later.
 Default installation only contains components not depending specific platforms (devices-simulators) or external libraries. You need to specify extras with square brackets to use those platforms and external libraries with QURI Parts:  Use `pip` to install QURI Parts.

### Quantum gates and circuits with QURI Parts
Quantum gates and circuits are essential when working on quantum computing. Here we describe basic treatment of them in QURI Parts. Here is what you need to do to start:

In [ ]:
!pip install "quri-parts[braket,cirq,qiskit,qulacs,tket]" 

### QuantumGate object
In QURI Parts, a quantum gate is represented by a `QuantumGate` object (more precisely `NamedTuple`). A `QuantumGate` contains not only the kind of the gate but also some additional information such as gate parameters and qubits on which the gate acts. You can create gate objects using `QuantumGate`:

In [ ]:
from math import pi 
from quri_parts.circuit import QuantumGate 
gates = [ 
# X gate acting on qubit 0 
QuantumGate("X", target_indices=(0,)), 
# Rotation gate acting on qubit 1 with angle pi/3 
QuantumGate("RX", target_indices=(1,), params=(pi/3,)), 
# CNOT gate on control qubit 2 and target qubit 1 
QuantumGate("CNOT", target_indices=(1,), control_indices=(2,)) 
] 
for gate in gates: 
    print(gate) 

### Interface
 When performing a sampling measurement for a circuit, you can use a `Sampler`. In QURI Parts, a `Sampler` represents a function that samples a specified (non-parametric) circuit by a specified times and returns the count statistics. In the case of an ideal `Sampler`, the return value corresponds to probabilities multiplied by shot count.:

In the case where sampling from multiple circuits is desired, QURI Parts also provide `ConcurrentSampler`, which is a function that samples from multiple (circuit, shot) pairs.

`Sampler` and `ConcurrentSampler` are both abstract interfaces with the following function signatures:

In [ ]:
from typing import Callable, Iterable, Mapping, Union
from typing_extensions import TypeAlias

from quri_parts.circuit import NonParametricQuantumCircuit

#: MeasurementCounts represents count statistics of repeated measurements of a quantum
#: circuit. Keys are observed bit patterns encoded in integers and values are counts
#: of observation of the corresponding bit patterns.
MeasurementCounts: TypeAlias = Mapping[int, Union[int, float]]
#: Sampler represents a function that samples a specified (non-parametric) circuit by
#: a specified times and returns the count statistics. In the case of an ideal Sampler,
# the return value corresponds to probabilities multiplied by shot count.
Sampler: TypeAlias = Callable[[NonParametricQuantumCircuit, int], MeasurementCounts]
#: ConcurrentSampler represents a function that samples specified (non-parametric)
#: circuits concurrently.
ConcurrentSampler: TypeAlias = Callable[
 [Iterable[tuple[NonParametricQuantumCircuit, int]]], Iterable[MeasurementCounts]
]

However it is more convenient to use built in factory functions:

In [ ]:
from quri_parts.circuit import X, RX, CNOT 
gates = [
# X gate acting on qubit 0
X(0),
# Rotation gate acting on qubit 1 with angle pi/3
RX(1, pi/3),
# CNOT gate on control qubit 2 and target qubit 1
CNOT(2, 1),
]
for gate in gates:
    print(gate)

In QURI Parts single-qubit rotation gates are defined as follows: <br>$R_X(\theta)= exp(-i\frac{\theta}{2}X)$ <br>$R_Y(\theta)= exp(-i\frac{\theta}{2}Y)$ <br>$R_Z(\theta)= exp(-i\frac{\theta}{2}Z)$ <br>where $\theta$ is called the angle of the gate.<br>You can access (but not set) attributes of a gate object:

In [2]:
from quri_parts.circuit import PauliRotation 
x_gate = X(0) 
print(f"name: {x_gate.name}, target: {x_gate.target_indices}")

rx_gate = RX(1, pi/3)
print(f"name: {rx_gate.name}, target: {rx_gate.target_indices}, angle: {rx_gate.params[0]}")

cnot_gate = CNOT(2, 1)
print(f"name: {cnot_gate.name}, control: {cnot_gate.control_indices}, target: {cnot_gate.target_indices}")

pauli_rot_gate = PauliRotation(target_indices=(0, 1, 2), pauli_ids=(1, 2, 3), angle=pi/3)
print(f"name: {pauli_rot_gate.name}, target: {pauli_rot_gate.target_indices}, pauli_ids: {pauli_rot_gate.pauli_ids}, angle: {pauli_rot_gate.params[0]}")

name: X, target: (0,)
name: RX, target: (1,), angle: 1.0471975511965976 
name: CNOT, control: (2,), target: (1,) 
name: PauliRotation, target: (0, 1, 2), pauli_ids: (1, 2, 3), angle: 1.0471975511965976

###QuantumCircuit object
You can construct a quantum circuit by specifying the number of qubits used in the circuit as follows:

In [ ]:
from quri_parts.circuit import QuantumCircuit 

# Create a circuit for 3 qubits 
 circuit = QuantumCircuit(3) 
# Add an already created QuantumGate object
circuit.add_gate(X(0)) 
# Or use methods to add gates 
circuit.add_X_gate(0)
circuit.add_RX_gate(1, pi/3)
circuit.add_CNOT_gate(2, 1)
circuit.add_PauliRotation_gate(target_qubits=(0, 1, 2), pauli_id_list=(1, 2, 3), angle=pi/3)

A `QuantumCircuit` object has several properties:

In [5]:
print("Qubit count:", circuit.qubit_count)
print("Circuit depth:", circuit.depth)

gates = circuit.gates # .gates returns the gates in the circuit as a sequence 
print("# of gates in the circuit:", len(gates))
for gate in gates:
    print(gate)


Qubit count: 3 
Circuit depth: 3 
# of gates in the circuit: 5 
QuantumGate(name='X', target_indices=(0,), control_indices=(), classical_indices=(), params=(), pauli_ids=(), unitary_matrix=())
QuantumGate(name='X', target_indices=(0,), control_indices=(), classical_indices=(), params=(), pauli_ids=(), unitary_matrix=())
QuantumGate(name='RX', target_indices=(1,), control_indices=(), classical_indices=(), params=(1.0471975511965976,), pauli_ids=(), unitary_matrix=())
QuantumGate(name='CNOT', target_indices=(1,), control_indices=(2,), classical_indices=(), params=(), pauli_ids=(), unitary_matrix=())
QuantumGate(name='PauliRotation', target_indices=(0, 1, 2), control_indices=(), classical_indices=(), params=(1.0471975511965976,), pauli_ids=(1, 2, 3), unitary_matrix=())

`QuantumCircuit` objects can be combined and extended:

In [5]:
circuit2 = QuantumCircuit(3)
circuit2.add_Y_gate(1)
circuit2.add_H_gate(2)

combined = circuit + circuit2 # equivalent: combined = circuit.combine(circuit2)
print("Combined circuit:", combined.gates) 
circuit2 += circuit # equivalent: circuit2.extend(circuit)
print("Extended circuit:", circuit2.gates)
# You can also embed a smaller circuit into a larger one
circuit_larger = QuantumCircuit(5)
circuit_larger.add_X_gate(3)
circuit_smaller = QuantumCircuit(3)
circuit_smaller.add_H_gate(0)
circuit_larger.extend(circuit_smaller)
print("Circuit extended by smaller one:", circuit_larger.gates)

Combined circuit:
(QuantumGate(name='X', target_indices=(0,), control_indices=(), classical_indices=(), params=(), pauli_ids=(), unitary_matrix=()), QuantumGate(name='X', target_indices=(0,), control_indices=(), classical_indices=(), params=(), pauli_ids=(), unitary_matrix=()), QuantumGate(name='RX', target_indices=(1,), control_indices=(), classical_indices=(), params=(1.0471975511965976,), pauli_ids=(), unitary_matrix=()), QuantumGate(name='CNOT', target_indices=(1,), control_indices=(2,), classical_indices=(), params=(), pauli_ids=(), unitary_matrix=()), QuantumGate(name='PauliRotation', target_indices=(0, 1, 2), control_indices=(), classical_indices=(), params=(1.0471975511965976,), pauli_ids=(1, 2, 3), unitary_matrix=()), QuantumGate(name='Y', target_indices=(1,), control_indices=(), classical_indices=(), params=(), pauli_ids=(), unitary_matrix=()), QuantumGate(name='H', target_indices=(2,), control_indices=(), classical_indices=(), params=(), pauli_ids=(), unitary_matrix=()))
Ext

## Sampler
Unlike statevector simulation, sampling measurements are necessary in order to estimate expectation value of operators on a quantum computer. In sampling measurements, execution of a quantum circuit and a subsequent measurement of qubits are repeated multiple times. Estimation of expectation value of operators is then performed using statistics of the repeated measurements. 

To perform a sampling measurement of a circuit, you can use a `Sampler`. Here we introduce the definition of `Sampler` and explain how it can be created or executed.

### Prerequisite
QURI Parts modules used in this tutorial: `quri-parts-circuit`, `quri-parts-core` and `quri-parts-qulacs`. You can install them as follows: 

In [2]:
!pip install "quri-parts[qulacs]"

### Prepare a circuit
As a preparation, we create a circuit to be sampled: 

In [2]:
from math import pi 
from quri_parts.circuit import QuantumCircuit 

# A circuit with 4 qubits
circuit = QuantumCircuit(4)
circuit.add_X_gate(0)
circuit.add_H_gate(1)
circuit.add_Y_gate(2)
circuit.add_CNOT_gate(1, 2)
circuit.add_RX_gate(3, pi/4)

### Interface
When performing a sampling measurement for a circuit, you can use a `Sampler`. In QURI Parts, a `Sampler` represents a function that samples a specified (non-parametric) circuit by a specified times and returns the count statistics. In the case of an ideal `Sampler`, the return value corresponds to probabilities multiplied by shot count. 

In the case where sampling from multiple circuits is desired, QURI Parts also provide `ConcurrentSampler`, which is a function that samples from multiple (circuit, shot) pairs. 
`Sampler` and `ConcurrentSampler` are both abstract interfaces with the following function signatures:

In [2]:
from typing import Callable, Iterable, Mapping, Union 
from typing_extensions import TypeAlias

from quri_parts.circuit import NonParametricQuantumCircuit

#: MeasurementCounts represents count statistics of repeated measurements of a quantum
#: circuit. Keys are observed bit patterns encoded in integers and values are counts
#: of observation of the corresponding bit patterns.
MeasurementCounts: TypeAlias = Mapping[int, Union[int, float]]

#: Sampler represents a function that samples a specified (non-parametric) circuit by
#: a specified times and returns the count statistics. In the case of an ideal Sampler,
# the return value corresponds to probabilities multiplied by shot count.
Sampler: TypeAlias = Callable[[NonParametricQuantumCircuit, int], MeasurementCounts]

#: ConcurrentSampler represents a function that samples specified (non-parametric)
#: circuits concurrently.
ConcurrentSampler: TypeAlias = Callable[
[Iterable[tuple[NonParametricQuantumCircuit, int]]], Iterable[MeasurementCounts]
]


The `Sampler` itself (defined in `quri_parts.core.sampling`) is an abstract interface and you need a concrete instance to actually perform sampling. There are several implementations of `Sampler` interface, some of which use a circuit simulator while others use a real quantum computer.

### Create and execute sampler
Let's create a sampler using state vector simulation with Qulacs and execute sampling with it.

In [2]:
from quri_parts.qulacs.sampler import create_qulacs_vector_sampler

# Create the sampler
sampler = create_qulacs_vector_sampler()
sampling_result = sampler(circuit, shots=1000)
print(sampling_result)

Counter({5: 423, 3: 419, 13: 90, 11: 68})

### Reference

[1] https://quri-parts-doc.netlify.app 